<h1 style="text-align: center"> Handling missing values for key ingredients using Gemini & Langchain

 </h1>

# 1. Importing the libraries 

In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 38.2 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade langchain

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai pillow


In [ ]:
pip install langchain_community

In [ ]:
pip install pypdf

In [ ]:
pip install docarray

In [ ]:
import pandas as pd
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import os
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from google.generativeai.types.safety_types import HarmCategory, HarmBlockThreshold
import pickle
import pandas as pd
from tqdm import tqdm

# 2. Setting up Gemini

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [ ]:
GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"]


In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
def to_markdown(text):
    # Replace bullet points with Markdown bullet points
    text = text.replace('•', '  *')
    # Indent the text with a greater-than symbol
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924


# 3. Handling the missing values

In [ ]:
model = genai.GenerativeModel(
    model_name='models/gemini-pro')

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-pro",temperature=0.9)


In [ ]:
prompt_template=''' You are a culinary assistant. When I provide the name of a dish, your task is to extract and list the key ingredients used in that dish. Please provide the key ingredients in a clear and concise manner, separating each ingredient with a comma.

Example Input:

Dish Name: Table-smashed Guacamole
Expected Output: lime, chilli, lime juice, plantain chips

Dish Name: {input_text}
Expected Output:
'''

In [ ]:
prompt = PromptTemplate.from_template(
    prompt_template
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
out=chain.run('MASH POTATO')

In [ ]:
out

'potatoes, milk, butter, salt, pepper'

In [ ]:
df=pd.read_csv('/content/missing_DF.csv')

In [ ]:

output=[]
for dish in tqdm(df.Dish,total=len(df)):
    out=chain.run(dish)

    output.append(out)

In [ ]:
pickle.dump(output,open('GenAIOutput.pkl','wb'))